# Object Detection

## Import everything

In [3]:
import os
import sys

import cv2
import numpy as np

from ote_sdk.configuration.helper import create as create_parameters_from_parameters_schema
from ote_sdk.entities.inference_parameters import InferenceParameters
from ote_sdk.entities.label_schema import LabelSchemaEntity
from ote_sdk.entities.model import ModelEntity, ModelStatus
from ote_sdk.entities.model_template import parse_model_template
from ote_sdk.entities.resultset import ResultSetEntity
from ote_sdk.entities.subset import Subset
from ote_sdk.entities.task_environment import TaskEnvironment
from ote_sdk.usecases.tasks.interfaces.export_interface import ExportType

from ote_cli.datasets import get_dataset_class
from ote_cli.registry import Registry
from ote_cli.utils.importing import get_impl_class

## Register templates

In [4]:
templates_dir = '../../external'
TYPE = "DETECTION"
#registry = Registry(templates_dir).filter(task_type=sys.executable.split(os.sep)[-4])
registry = Registry(templates_dir).filter(task_type=TYPE)
print(registry)

- id: Custom_Object_Detection_Gen3_ATSS
  name: ATSS
  path: /home/mau/intel/training/training_extensions/external/mmdetection/configs/ote/custom-object-detection/gen3_mobilenetV2_ATSS/template.yaml
  task_type: DETECTION
- id: Custom_Object_Detection_Gen3_SSD
  name: SSD
  path: /home/mau/intel/training/training_extensions/external/mmdetection/configs/ote/custom-object-detection/gen3_mobilenetV2_SSD/template.yaml
  task_type: DETECTION



## Load model template and its hyper parameters

In [5]:
model_template = registry.get('Custom_Object_Detection_Gen3_ATSS')
hyper_parameters = model_template.hyper_parameters.data
print(type(model_template), "\n", type(hyper_parameters))

<class 'ote_sdk.entities.model_template.ModelTemplate'> 
 <class 'dict'>


## Get dataset instantiated

In [6]:
Dataset = get_dataset_class(model_template.task_type)

dataset = Dataset(
    train_subset={'ann_file': '../../data/airport/annotation_faces_train.json',
                  'data_root': '../../data/airport/'},
    val_subset={'ann_file': '../../data/airport/annotation_faces_train.json',
                'data_root': '../../data/airport'}
)
labels_schema = LabelSchemaEntity.from_labels(dataset.get_labels())

No CUDA runtime is found, using CUDA_HOME='/usr'
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


## Have a look at existing parameters

In [7]:
hyper_parameters = create_parameters_from_parameters_schema(hyper_parameters)

for p in hyper_parameters.learning_parameters.parameters:
    print(f'{p}: {getattr(hyper_parameters.learning_parameters, p)}')

batch_size: 8
learning_rate: 0.008
learning_rate_warmup_iters: 200
num_iters: 300
num_workers: 4


## Tweak parameters

In [8]:
hyper_parameters.learning_parameters.batch_size = 8
hyper_parameters.learning_parameters.num_iters = 5

for p in hyper_parameters.learning_parameters.parameters:
    print(f'{p}: {getattr(hyper_parameters.learning_parameters, p)}')

batch_size: 8
learning_rate: 0.008
learning_rate_warmup_iters: 200
num_iters: 5
num_workers: 4


## Create Task 

In [9]:
Task = get_impl_class(model_template.entrypoints.base)

environment = TaskEnvironment(
    model=None,
    hyper_parameters=hyper_parameters,
    label_schema=labels_schema,
    model_template=model_template)
        
task = Task(task_environment=environment)

15:01:58 accuracy_checker WARNING: /home/mau/intel/training/training_extensions/det_venv/lib/python3.8/site-packages/defusedxml/__init__.py:30: DeprecationWarning: defusedxml.cElementTree is deprecated, import from defusedxml.ElementTree instead.
  from . import cElementTree

15:01:58 accuracy_checker WARNING: /home/mau/intel/training/training_extensions/det_venv/lib/python3.8/site-packages/compression/algorithms/quantization/optimization/algorithm.py:29: UserWarning: Nevergrad package could not be imported. If you are planning to useany hyperparameter optimization algo, consider installing itusing pip. This implies advanced usage of the tool.Note that nevergrad is compatible only with Python 3.6+
  warnings.warn(

2022-08-22 15:01:58,898 - mmdet - INFO - Loading OTEDetectionTask


[ INFO ] Loading OTEDetectionTask


[ INFO ]  Loading OTEDetectionTask


ENVIRONMENT:
sys.platform: linux
Python: 3.8.10 (default, Jun 22 2022, 20:18:18) [GCC 9.4.0]
CUDA available: False
GCC: gcc (Ubuntu 9.4.0-1ubuntu1~20.04.1) 9.4.0
PyTorch: 1.8.2+cpu
PyTorch compiling details: PyTorch built with:
  - GCC 7.3
  - C++ Version: 201402
  - Intel(R) Math Kernel Library Version 2020.0.0 Product Build 20191122 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v1.7.0 (Git Hash 7aed236906b1f7a05c0917e5257a1af05e9ff683)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - Build settings: BLAS_INFO=mkl, BUILD_TYPE=Release, CXX_COMPILER=/opt/rh/devtoolset-7/root/usr/bin/c++, CXX_FLAGS= -Wno-deprecated -fvisibility-inlines-hidden -DUSE_PTHREADPOOL -fopenmp -DNDEBUG -DUSE_FBGEMM -DUSE_QNNPACK -DUSE_PYTORCH_QNNPACK -DUSE_XNNPACK -O2 -fPIC -Wno-narrowing -Wall -Wextra -Werror=return-type -Wno-missing-field-initializers -Wno-type-limits -Wno-array-bounds -Wno-unknown-pragmas -Wno-sign-compare -Wno-unused-parameter -W

You should consider upgrading via the '/home/mau/intel/training/training_extensions/det_venv/bin/python3 -m pip install --upgrade pip' command.
2022-08-22 15:01:59,355 - mmdet - INFO - Scratch space created at /tmp/ote-det-scratch-_g6qhbxh


...

[ WARNING ]  load checkpoint
2022-08-22 15:01:59,508 - mmdet - INFO - No trained model in project yet. Created new model with 'ATSS' architecture and general-purpose pretrained weights.


Use load_from_http loader
The model and loaded state dict do not match exactly

size mismatch for bbox_head.atss_cls.weight: copying a param with shape torch.Size([80, 64, 3, 3]) from checkpoint, the shape in current model is torch.Size([1, 64, 3, 3]).
size mismatch for bbox_head.atss_cls.bias: copying a param with shape torch.Size([80]) from checkpoint, the shape in current model is torch.Size([1]).
[ INFO ] No trained model in project yet. Created new model with 'ATSS' architecture and general-purpose pretrained weights.


[ INFO ]  No trained model in project yet. Created new model with 'ATSS' architecture and general-purpose pretrained weights.
2022-08-22 15:01:59,509 - mmdet - INFO - Task initialization completed


[ INFO ] Task initialization completed


[ INFO ]  Task initialization completed


## Run training

In [ ]:
output_model = ModelEntity(
    dataset,
    environment.get_model_configuration(),
    model_status=ModelStatus.NOT_READY)

task.train(dataset, output_model)

2022-08-22 15:02:06,837 - mmdet - INFO - Training the model


[ INFO ] Training the model


[ INFO ]  Training the model
2022-08-22 15:02:06,866 - mmdet - INFO - Checkpoints and logs for this training run are stored in /tmp/ote-det-scratch-_g6qhbxh/checkpoints_round_0


[ INFO ] Checkpoints and logs for this training run are stored in /tmp/ote-det-scratch-_g6qhbxh/checkpoints_round_0


[ INFO ]  Checkpoints and logs for this training run are stored in /tmp/ote-det-scratch-_g6qhbxh/checkpoints_round_0
2022-08-22 15:02:06,869 - mmdet - INFO - Start training


...

[ INFO ]  Start running, host: %s, work_dir: %s
2022-08-22 15:02:06,898 - mmdet - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) ReduceLROnPlateauLrUpdaterHook     
(NORMAL      ) CheckpointHook                     
(NORMAL      ) EvalHook                           
(NORMAL      ) OTEProgressHook                    
(75          ) EarlyStoppingHook                  
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) OTELoggerHook                      
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) ReduceLROnPlateauLrUpdaterHook     
(NORMAL      ) EvalHook                           
(NORMAL      ) OTEProgressHook                    
(LOW         ) IterTimerHook                      
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) OTELoggerHook                      
 -------------------- 
before_train_iter:
(VERY_HIGH   ) ReduceLROnPlateauLrUpdaterHook     
(NORMAL      ) EvalHook                           
(N

[ INFO ] Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) ReduceLROnPlateauLrUpdaterHook     
(NORMAL      ) CheckpointHook                     
(NORMAL      ) EvalHook                           
(NORMAL      ) OTEProgressHook                    
(75          ) EarlyStoppingHook                  
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) OTELoggerHook                      
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) ReduceLROnPlateauLrUpdaterHook     
(NORMAL      ) EvalHook                           
(NORMAL      ) OTEProgressHook                    
(LOW         ) IterTimerHook                      
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) OTELoggerHook                      
 -------------------- 
before_train_iter:
(VERY_HIGH   ) ReduceLROnPlateauLrUpdaterHook     
(NORMAL      ) EvalHook                           
(NORMAL      ) OTEProgressHook                    
(LOW         ) IterTimerHook   

[ INFO ]  Hooks will be executed in the following order:
%s
2022-08-22 15:02:06,898 - mmdet - INFO - workflow: [('train', 1)], max: 5 epochs


[ INFO ] workflow: [('train', 1)], max: 5 epochs


[ INFO ]  workflow: %s, max: %d epochs


## Evaluate quality metric

In [1]:
validation_dataset = dataset.get_subset(Subset.VALIDATION)
predicted_validation_dataset = task.infer(
    validation_dataset.with_empty_annotations(),
    InferenceParameters(is_evaluation=True))

resultset = ResultSetEntity(
    model=output_model,
    ground_truth_dataset=validation_dataset,
    prediction_dataset=predicted_validation_dataset,
)
task.evaluate(resultset)
print(resultset.performance)


NameError: name 'dataset' is not defined

## Export model to OpenVINO format

In [ ]:
exported_model = ModelEntity(
    dataset,
    environment.get_model_configuration(),
    model_status=ModelStatus.NOT_READY)
task.export(ExportType.OPENVINO, exported_model)

## Evaluate the exported model

In [ ]:
environment.model = exported_model
ov_task = get_impl_class(model_template.entrypoints.openvino)(environment)
predicted_validation_dataset = ov_task.infer(
    validation_dataset.with_empty_annotations(),
    InferenceParameters(is_evaluation=True))
resultset = ResultSetEntity(
    model=output_model,
    ground_truth_dataset=validation_dataset,
    prediction_dataset=predicted_validation_dataset,
)
ov_task.evaluate(resultset)
print(resultset.performance)

## Draw bounding boxes

In [ ]:
import IPython
import PIL

for predictions, item in zip(predicted_validation_dataset, validation_dataset.with_empty_annotations()):
    image = item.numpy.astype(np.uint8)
    for box in predictions.annotation_scene.shapes:
        x1 = int(box.x1 * image.shape[1])
        x2 = int(box.x2 * image.shape[1])
        y1 = int(box.y1 * image.shape[0])
        y2 = int(box.y2 * image.shape[0])
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 0, 255), 3)  
    IPython.display.display(PIL.Image.fromarray(image))